In [1]:
from baybe import Campaign
from baybe.objectives import SingleTargetObjective
from baybe.parameters import NumericalContinuousParameter, NumericalDiscreteParameter, CategoricalParameter, TaskParameter
from baybe.searchspace import SearchSpace
from baybe.targets import NumericalTarget
from baybe.constraints import ContinuousLinearInequalityConstraint
from baybe.constraints import DiscreteSumConstraint, ThresholdCondition
from baybe.simulation import simulate_scenarios

import numpy as np
import pandas as pd
import os
import seaborn as sns
import torch
# load the Advanced Optimization from AC huggingface
from gradio_client import Client
client = Client("AccelerationConsortium/crabnet-hyperparameter")

from baybe.utils.random import set_random_seed

Loaded as API: https://accelerationconsortium-crabnet-hyperparameter.hf.space ✔


In [2]:
DIMENSION = 3  # input dimensionality of the test function
BATCH_SIZE = 1  # batch size of recommendations per DOE iteration
N_MC_ITERATIONS = 5
N_DOE_ITERATIONS = 30

In [3]:
# y1 and y2 are correlated 
# Adv Opt function for y1
def adv_opt_y1(x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, x16, x17, x18, x19, x20, c1, c2, c3): 
    result = client.predict(
        x1, # float (numeric value between 0.0 and 1.0) in 'x1' Slider component
		x2,	# float (numeric value between 0.0 and 1.0)	in 'x2' Slider component
		x3,	# float (numeric value between 0.0 and 1.0) in 'x3' Slider component
		x4,	# float (numeric value between 0.0 and 1.0) in 'x4' Slider component
		x5,	# float (numeric value between 0.0 and 1.0) in 'x5' Slider component
		x6,	# float (numeric value between 0.0 and 1.0) in 'x6' Slider component
		x7,	# float (numeric value between 0.0 and 1.0) in 'x7' Slider component
		x8,	# float (numeric value between 0.0 and 1.0) in 'x8' Slider component
		x9,	# float (numeric value between 0.0 and 1.0) in 'x9' Slider component
		x10,	# float (numeric value between 0.0 and 1.0) in 'x10' Slider component
		x11,	# float (numeric value between 0.0 and 1.0) in 'x11' Slider component
		x12,	# float (numeric value between 0.0 and 1.0) in 'x12' Slider component
		x13,	# float (numeric value between 0.0 and 1.0) in 'x13' Slider component
		x14,	# float (numeric value between 0.0 and 1.0) in 'x14' Slider component
		x15,	# float (numeric value between 0.0 and 1.0) in 'x15' Slider component
		x16,	# float (numeric value between 0.0 and 1.0) in 'x16' Slider component
		x17,	# float (numeric value between 0.0 and 1.0) in 'x17' Slider component
		x18,	# float (numeric value between 0.0 and 1.0) in 'x18' Slider component
		x19,	# float (numeric value between 0.0 and 1.0) in 'x19' Slider component
		x20,	# float (numeric value between 0.0 and 1.0) in 'x20' Slider component
		c1,	# Literal['c1_0', 'c1_1'] in 'c1' Radio component
		c2,	# Literal['c2_0', 'c2_1'] in 'c2' Radio component
		c3,	# Literal['c3_0', 'c3_1', 'c3_2'] in 'c3' Radio component
		0.5,	# float (numeric value between 0.0 and 1.0) in 'fidelity1' Slider component
		api_name="/predict",
    )
    return result['data'][0][0]			# return y1 value only 

# Adv Opt function for y2
def adv_opt_y2(x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, x16, x17, x18, x19, x20, c1, c2, c3): 
    result = client.predict(
        x1, # float (numeric value between 0.0 and 1.0) in 'x1' Slider component
		x2,	# float (numeric value between 0.0 and 1.0)	in 'x2' Slider component
		x3,	# float (numeric value between 0.0 and 1.0) in 'x3' Slider component
		x4,	# float (numeric value between 0.0 and 1.0) in 'x4' Slider component
		x5,	# float (numeric value between 0.0 and 1.0) in 'x5' Slider component
		x6,	# float (numeric value between 0.0 and 1.0) in 'x6' Slider component
		x7,	# float (numeric value between 0.0 and 1.0) in 'x7' Slider component
		x8,	# float (numeric value between 0.0 and 1.0) in 'x8' Slider component
		x9,	# float (numeric value between 0.0 and 1.0) in 'x9' Slider component
		x10,	# float (numeric value between 0.0 and 1.0) in 'x10' Slider component
		x11,	# float (numeric value between 0.0 and 1.0) in 'x11' Slider component
		x12,	# float (numeric value between 0.0 and 1.0) in 'x12' Slider component
		x13,	# float (numeric value between 0.0 and 1.0) in 'x13' Slider component
		x14,	# float (numeric value between 0.0 and 1.0) in 'x14' Slider component
		x15,	# float (numeric value between 0.0 and 1.0) in 'x15' Slider component
		x16,	# float (numeric value between 0.0 and 1.0) in 'x16' Slider component
		x17,	# float (numeric value between 0.0 and 1.0) in 'x17' Slider component
		x18,	# float (numeric value between 0.0 and 1.0) in 'x18' Slider component
		x19,	# float (numeric value between 0.0 and 1.0) in 'x19' Slider component
		x20,	# float (numeric value between 0.0 and 1.0) in 'x20' Slider component
		c1,	# Literal['c1_0', 'c1_1'] in 'c1' Radio component
		c2,	# Literal['c2_0', 'c2_1'] in 'c2' Radio component
		c3,	# Literal['c3_0', 'c3_1', 'c3_2'] in 'c3' Radio component
		0.5,	# float (numeric value between 0.0 and 1.0) in 'fidelity1' Slider component
		api_name="/predict",
    )
    return result['data'][0][1]			# return y2 value only

In [4]:
# define and create the search space
parameters = [
    NumericalContinuousParameter(name="x1", bounds=(0.0, 1.0)), 
    NumericalContinuousParameter(name="x2", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x3", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x4", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x5", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x6", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x7", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x8", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x9", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x10", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x11", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x12", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x13", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x14", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x15", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x16", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x17", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x18", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x19", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x20", bounds=(0.0, 1.0)),

    CategoricalParameter(name='c1', values=['c1_0', 'c1_1'], encoding="OHE"),
    CategoricalParameter(name='c2', values=['c2_0', 'c2_1'], encoding="OHE"),
    CategoricalParameter(name='c3', values=['c3_0', 'c3_1', 'c3_2'], encoding="OHE"),
]

task_parameters = TaskParameter(name="Function", 
                                values = ["TrainingY2", "TestingY1"], 
                                active_values=["TestingY1"]
                                )

constraints = [
    ContinuousLinearInequalityConstraint(parameters=["x19", "x20"], coefficients=[-1.0, 1.0], rhs=0.0),
    ContinuousLinearInequalityConstraint(parameters=["x6", "x15"], coefficients=[-1.0, -1.0], rhs=-1.0), 
]

params = [*parameters, task_parameters]
searchspace = SearchSpace.from_product(parameters=params, constraints=constraints)

# define objective
objective = SingleTargetObjective(target=NumericalTarget(name="Target", mode="MIN"))

In [5]:
# define the tasks
# consider optimizing y1 using training data from y2
test_functions = {
    "TrainingY2": adv_opt_y2,
    "TestingY1": adv_opt_y1,
}

In [6]:
# # Define initial data  --------------------
# def generate_parameters():
#     while True:
#         # Random float values for x1 to x20 between 0.0 and 1.0
#         params = {f"x{i}": np.random.uniform(0.0, 1.0) for i in range(1, 21)}
        
#         # Random categorical values for c1, c2, c3
#         params["c1"] = np.random.choice(["c1_0", "c1_1"])
#         params["c2"] = np.random.choice(["c2_0", "c2_1"])
#         params["c3"] = np.random.choice(["c3_0", "c3_1", "c3_2"])
        
#         # Check constraints
#         if params["x19"] < params["x20"] and params["x6"] + params["x15"] <= 1.0:
#             return params

# # Create DataFrame for each set size in number_init_points
# data = [generate_parameters() for _ in range(20)]
# initial_points = pd.DataFrame(data)
# # make sure c1, c2, c3 are str type
# initial_points['c1'] = initial_points['c1'].apply(str)
# initial_points['c2'] = initial_points['c2'].apply(str)
# initial_points['c3'] = initial_points['c3'].apply(str)
# initial_points

In [7]:
# # create a dataframe, that has initial points for y1 and y2
# # add a Target column for y1/y2 value, and a Function column for the fucntion used 
# lookup_training_y2 = initial_points.copy()
# lookup_training_y2['Target'] = lookup_training_y2.apply(lambda x: adv_opt_y2(**x), axis=1)
# lookup_training_y2['Function'] = "TrainingY2"

# lookup_testing_y1 = initial_points.copy()
# lookup_testing_y1['Target'] = lookup_testing_y1.apply(lambda x: adv_opt_y1(**x), axis=1)
# lookup_testing_y1['Function'] = "TestingY1"

# # save lookup_training_y2 and lookup_testing_y1 to csv
# lookup_testing_y1.to_csv("BayBE_lookup_testing_y1.csv", index=False)
# lookup_training_y2.to_csv("BayBE_lookup_training_y2.csv", index=False)

lookup_training_y2 = pd.read_csv("BayBE_lookup_training_y2.csv")
lookup_testing_y1 = pd.read_csv("BayBE_lookup_testing_y1.csv")

In [8]:
import time
random_seed_list = [23, 42, 87, 131, 518]
round = 30
results = pd.DataFrame()
for i in range(len(random_seed_list)): 
    set_random_seed(random_seed_list[i])

    # create a campaign
    campaign = Campaign(searchspace=searchspace, objective=objective)

    # set initial data
    init_df = lookup_training_y2.iloc[:20]       # ---------------- change here for different initial data size [1, 3, 5, 20]]

    # add initial data to campaign
    campaign.add_measurements(init_df)

    for k in range(round):
        recommendation = campaign.recommend(batch_size=1)
        # select the numerical columns
        numerical_cols = recommendation.select_dtypes(include='number')
        # replace values less than 1e-8 with 0 in numerical columns
        numerical_cols = numerical_cols.map(lambda x: 0 if x < 1e-6 else x)
        # update the original DataFrame
        recommendation.update(numerical_cols)
        
        # if x6+x15 >1.0, round x6 and x15 to 4 decimal places
        while recommendation['x6'].item() + recommendation['x15'].item() > 1.0: 
            recommendation['x6'] = np.round(recommendation['x6'].item(), 4)
            recommendation['x15'] = np.round(recommendation['x15'].item(), 4)

        # if x19 >= x20, subtract 1e-6 from x19 and add 1e-6 to x20
        while recommendation['x19'].item() >= recommendation['x20'].item():
            recommendation['x19'] = recommendation['x19'].item() - 1e-6
            # if recommendation['x19'] < 0, assign 0 to x19
            if recommendation['x19'].item() < 0:
                recommendation['x19'] = 0
            recommendation['x20'] = recommendation['x20'].item() + 1e-6
            if recommendation['x20'].item() > 1:
                recommendation['x20'] = 1
       
        # target value are looked up via the botorch wrapper
        target_values = []
        for index, row in recommendation.iterrows():
            # print(row.to_dict())
            dict = row.to_dict()
            dict.pop('Function', None)
            target_values.append(adv_opt_y1(**dict))

        recommendation['Target'] = target_values
        time.sleep(15)
        campaign.add_measurements(recommendation)

    results = pd.concat([results, campaign.measurements])

c:\Users\MF\anaconda3\envs\BayBE\lib\site-packages\linear_operator\utils\interpolation.py:71: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:623.)
  summing_matrix = cls(summing_matrix_indices, summing_matrix_values, size)


In [9]:
results

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x18,x19,x20,c1,c2,c3,Target,Function,BatchNr,FitNr
0,0.868357,0.962026,0.121573,0.080041,0.795165,0.212426,0.357139,0.123713,0.864067,0.934670,...,0.135703,0.492192,0.800648,c1_0,c2_1,c3_0,0.958119,TrainingY2,1,1.0
1,0.944487,0.256564,0.537634,0.568320,0.787072,0.680124,0.973868,0.370550,0.865335,0.127380,...,0.395685,0.634365,0.775068,c1_0,c2_1,c3_1,0.907501,TrainingY2,1,1.0
2,0.683406,0.140660,0.189235,0.678551,0.055121,0.331954,0.734186,0.872262,0.019960,0.850660,...,0.339760,0.008893,0.556985,c1_1,c2_0,c3_0,0.940497,TrainingY2,1,1.0
3,0.573291,0.745384,0.459138,0.645285,0.244274,0.013635,0.628135,0.622687,0.737590,0.893893,...,0.973161,0.262136,0.561262,c1_0,c2_0,c3_0,0.990021,TrainingY2,1,1.0
4,0.140547,0.556007,0.975023,0.780218,0.211473,0.182598,0.707216,0.829014,0.691934,0.780679,...,0.760250,0.166208,0.580540,c1_1,c2_1,c3_0,0.927315,TrainingY2,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.186574,1.000000,0.000000,...,0.000000,0.000000,0.000001,c1_0,c2_0,c3_2,0.409993,TestingY1,27,27.0
46,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,...,0.000000,0.999999,1.000000,c1_1,c2_0,c3_0,0.503114,TestingY1,28,28.0
47,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.199739,1.000000,0.000000,...,0.000000,0.000000,0.000001,c1_0,c2_0,c3_2,0.477075,TestingY1,29,29.0
48,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000001,c1_0,c2_1,c3_0,0.778660,TestingY1,30,30.0


In [11]:
# select the TestingY1 in results 
results_testing_y1 = results[results['Function'] == 'TestingY1']
results_testing_y1

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x18,x19,x20,c1,c2,c3,Target,Function,BatchNr,FitNr
20,0.805749,0.730909,0.795658,0.601083,0.922543,0.485718,0.583327,0.506131,0.680550,0.464909,...,0.871474,0.440056,0.936603,c1_0,c2_1,c3_0,0.415561,TestingY1,2,2.0
21,0.527101,1.000000,0.889699,0.423932,0.936101,0.681044,0.904073,0.663284,0.737414,0.209284,...,0.115158,0.345189,0.668912,c1_0,c2_0,c3_1,0.336896,TestingY1,3,3.0
22,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,...,1.000000,0.000000,1.000000,c1_0,c2_0,c3_0,0.590465,TestingY1,4,4.0
23,0.589060,1.000000,1.000000,0.119889,1.000000,1.000000,1.000000,0.239857,1.000000,0.000000,...,0.000000,0.132366,0.132366,c1_0,c2_1,c3_2,0.398579,TestingY1,5,5.0
24,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,...,1.000000,0.999999,1.000000,c1_0,c2_1,c3_2,0.721896,TestingY1,6,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.186574,1.000000,0.000000,...,0.000000,0.000000,0.000001,c1_0,c2_0,c3_2,0.409993,TestingY1,27,27.0
46,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,...,0.000000,0.999999,1.000000,c1_1,c2_0,c3_0,0.503114,TestingY1,28,28.0
47,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.199739,1.000000,0.000000,...,0.000000,0.000000,0.000001,c1_0,c2_0,c3_2,0.477075,TestingY1,29,29.0
48,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000001,c1_0,c2_1,c3_0,0.778660,TestingY1,30,30.0


In [12]:
# save results to csv
results_testing_y1.to_csv("init_20_BayBE_5round.csv", index=False)

In [13]:
test = pd.read_csv("init_20_BayBE_5round.csv")
test

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x18,x19,x20,c1,c2,c3,Target,Function,BatchNr,FitNr
0,0.805749,0.730909,0.795658,0.601083,0.922543,0.485718,0.583327,0.506131,0.680550,0.464909,...,0.871474,0.440056,0.936603,c1_0,c2_1,c3_0,0.415561,TestingY1,2,2.0
1,0.527101,1.000000,0.889699,0.423932,0.936101,0.681044,0.904073,0.663284,0.737414,0.209284,...,0.115158,0.345189,0.668912,c1_0,c2_0,c3_1,0.336896,TestingY1,3,3.0
2,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,...,1.000000,0.000000,1.000000,c1_0,c2_0,c3_0,0.590465,TestingY1,4,4.0
3,0.589060,1.000000,1.000000,0.119889,1.000000,1.000000,1.000000,0.239857,1.000000,0.000000,...,0.000000,0.132366,0.132366,c1_0,c2_1,c3_2,0.398579,TestingY1,5,5.0
4,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,...,1.000000,0.999999,1.000000,c1_0,c2_1,c3_2,0.721896,TestingY1,6,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.186574,1.000000,0.000000,...,0.000000,0.000000,0.000001,c1_0,c2_0,c3_2,0.409993,TestingY1,27,27.0
146,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,...,0.000000,0.999999,1.000000,c1_1,c2_0,c3_0,0.503114,TestingY1,28,28.0
147,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.199739,1.000000,0.000000,...,0.000000,0.000000,0.000001,c1_0,c2_0,c3_2,0.477075,TestingY1,29,29.0
148,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000001,c1_0,c2_1,c3_0,0.778660,TestingY1,30,30.0
